In [2]:
import sys
sys.path.append('../src')
import numpy as np

# Main Problems in Quantum Information

## 1. Motivation and Key Questions

- Boundary of entanglement
- Entanglement measures and classification (concurrence, entropy, negativity, etc.)
- Reversibility and noise tolerance of quantum channels
- Resource theory boundaries (magic, coherence, non-Gaussianity)
- Quantum vs classical complexity
- Bell inequalities and nonlocality

---

## 2. Goal

**[Completed]**
- Understand the mathematical and physical meaning of witnesses and the separable cone.
- Automate PPT witness generation via SDP.
- Compute violation for all bipartitions in multipartite states.
- Build a scalable numerical pipeline for future extensions (e.g., product state sampling, DPS hierarchy).

---

## 3. Background

**[Completed]**
- Separable states: classical-quantum boundary.
- Entanglement boundary: definition and significance.
- Physical and application meaning.
- Tools: PPT criterion, entanglement witness, DPS hierarchy, Bell inequalities, etc.

---

## 4. PPT Criterion

**[Completed]**
- Principle and implementation.
- Example: separable state vs Bell state.

```python
# Example: PPT criterion for separable and Bell states

## Separable States and the Entanglement Boundary

A separable state defines the boundary between classical and quantum correlations. A bipartite quantum state is separable if it can be written as a classical mixture of product states:
$$
\sigma = \sum_i p_i \, \rho_A^{(i)} \otimes \rho_B^{(i)}
$$
All correlations in separable states can be explained by classical probability, with no genuine quantum entanglement. In contrast, entangled states possess purely quantum correlations, which serve as a resource in quantum information science.

The set of separable states forms the baseline for entanglement—any state outside this set is considered entangled. Many tools in quantum information, such as entanglement witnesses and entanglement measures, are defined relative to the set of separable states. Clearly drawing this boundary is essential for determining whether a state possesses genuine quantum entanglement.

**Physical and Application Significance:**
- **Experiment:** To demonstrate that a state is entangled, one must rule out the possibility that it is merely separable.
- **Security:** In quantum cryptography, only entangled states can guarantee the security of certain protocols.
- **Resource Theory:** Separable states are considered "free" (costless), while entanglement is a valuable resource for quantum tasks.

### Main Tools for Entanglement Detection and Entanglement Boundaries

- **PPT (Positive Partial Transpose) Criterion:** The simplest and most widely used numerical tool. For 2x2 and 2x3 systems, it is both necessary and sufficient for separability; for higher dimensions, it is only necessary. The PPT cone is a convex set containing all separable states and some entangled states (bound entanglement).
- **Entanglement Witness:** A Hermitian operator designed as a "lie detector" for specific states, able to detect some entangled states. Can be constructed automatically via semidefinite programming (SDP). Witnesses are designed to distinguish between the separable cone and its relaxations (e.g., PPT cone).
- **DPS Hierarchy (Doherty-Parrilo-Spedalieri):** A sequence of increasingly strict SDP relaxations that approximate the separable cone. Each level ($k$-extendibility) imposes stronger constraints; as $k \to \infty$, the hierarchy converges to the true separable set. Computational cost grows rapidly with $k$.
- **Product State Sampling:** For large or multipartite systems, one can numerically sample many random product states $|\psi_A\rangle \otimes |\psi_B\rangle$ and require $W$ to be positive on these. This provides a practical, though not rigorous, numerical test for entanglement.
- **Bell Inequalities and Nonlocality Tests:** Used to test for nonlocality, which indirectly proves entanglement. Not all entangled states violate a Bell inequality.
- **Advanced Criteria (Tensor Rank, Realignment, CCNR, etc.):** Various mathematical tools providing necessary/sufficient conditions for entanglement in different scenarios.

These tools are often used in combination to characterize the boundary between separable and entangled states, both theoretically and numerically. In practice, relaxations like the PPT cone or DPS hierarchy are used for numerical detection, since the separable cone cannot be described by simple SDP constraints.

### Entanglement Witness

An **entanglement witness** is a Hermitian operator $W$ constructed to detect entanglement in a quantum state $\rho$.

- For all separable states $\sigma$, $\mathrm{Tr}[W\sigma] \geq 0$.
- For at least one entangled state $\rho$, $\mathrm{Tr}[W\rho] < 0$.

Intuitively, $W$ acts as a "lie detector": it never signals for honest (separable) states, but gives a negative value for some entangled (lying) states.

Witnesses can be constructed analytically for specific states, or automatically via semidefinite programming (SDP).

---

### The Separable Cone and PPT Cone

- The set of all separable states forms a **convex cone** in the space of density matrices, but its mathematical structure is highly complex.
- The **PPT cone** (Positive Partial Transpose) is a larger, simpler set that contains all separable states and some entangled states (bound entanglement). It is efficiently characterized by SDP constraints.
- In practice, relaxations like the PPT cone or DPS hierarchy are used for numerical detection, since the separable cone cannot be described by simple SDP constraints.

Summary:
- **Separable cone:** All non-entangled states; complex shape, hard to describe by SDP.
- **PPT cone:** Contains the separable cone; simple SDP description.
- **Witness $W$:** Designed to distinguish between these sets and numerically detect entanglement.

### Mathematical Formulation and SDP Construction

To construct an optimal entanglement witness for a target state $\rho$ using SDP:
- $W$ is Hermitian.
- $\mathrm{Tr}[W] = 1$ (normalization, optional).
- For all separable states $\sigma$, $\mathrm{Tr}[W\sigma] \geq 0$.
- For the target $\rho$, $\mathrm{Tr}[W\rho]$ as small as possible (ideally $<0$).

Since the set of separable states is hard to describe, we often relax the constraint to require $W$ to be positive on the PPT cone (i.e., $W^{T_B} \succeq 0$), or use higher levels of the DPS hierarchy. This leads to a semidefinite program (SDP):
- Minimize $\mathrm{Tr}[W\rho]$
- Subject to: $W$ Hermitian, $\mathrm{Tr}[W]=1$, $W^{T_B} \succeq 0$ (or other relaxations)

If the minimum is negative, $\rho$ is certified to be entangled (outside the chosen cone). This approach can be automated using SDP solvers such as CVXPY.

**Summary of SDP Workflow:**
1. Choose a relaxation cone (PPT or DPS $k$-extendible cone).
2. Write SDP constraints for the cone (e.g., partial transpose, extension symmetry).
3. Design the witness SDP (find $W$ in the dual cone minimizing $\mathrm{Tr}[W\rho]$).
4. Solve numerically (cvxpy, QETLAB, etc.).
5. If witness expectation $<0$, $\rho$ is entangled.

---

### Beyond PPT: DPS Hierarchy and Product State Sampling

- **DPS Hierarchy (Doherty-Parrilo-Spedalieri):**
  A sequence of increasingly strict SDP relaxations that approximate the separable cone. Each level ($k$-extendibility) imposes stronger constraints; as $k \to \infty$, the hierarchy converges to the true separable set. Computational cost grows rapidly with $k$.
- **Product State Sampling (Practical Approximation):**
  For large or multipartite systems, one can numerically sample many random product states $|\psi_A\rangle \otimes |\psi_B\rangle$ and require $W$ to be positive on these. This provides a practical, though not rigorous, numerical test for entanglement.

**Summary:**
- PPT-based SDP is the standard tool for small systems.
- DPS hierarchy gives stronger (but more expensive) relaxations.
- Product state sampling is useful for large/multipartite systems.

For practical implementation, see the code examples below using CVXPY.

### 1. PPT(Peres–Horodecki criterion) Criterion
#### (a) Two-body system（seperable state、Bell state）

In [3]:
from entanglement_analysis import is_separable_PPT

# Pauli projectors
P0 = np.array([[1,0],[0,0]])
P1 = np.array([[0,0],[0,1]])

# seperable state
rho_sep = 0.5 * np.kron(P0, P0) + 0.5 * np.kron(P1, P1)
# Bell state
psi_ent = (np.kron([1,0],[1,0]) + np.kron([0,1],[0,1]))/np.sqrt(2)
rho_ent = np.outer(psi_ent, psi_ent.conj())

dims = [2,2]

print("seperable state PPT:", is_separable_PPT(rho_sep, dims))  # should be True
print("Bell state PPT:", is_separable_PPT(rho_ent, dims))   # should be False

seperable state PPT: True
Bell state PPT: False


#### (b) Many-body system（GHZ、Cluster、Graph state）

In [4]:
from state_utils import ghz_state,cluster_state, graph_state
from entanglement_analysis import all_bipartition_PPT_checks, is_genuine_multipartite_entangled

N = 3
psi = ghz_state(N)
rho = np.outer(psi, psi.conj())

ppt_results = all_bipartition_PPT_checks(rho, dims=[2]*N)
print("Results of PPT criteria for each partition of GHZ state ：")

for part, is_ppt in ppt_results.items():
    print(f"Partition {part}: PPT = {is_ppt}")
    
N = 4
psi = cluster_state(N)
rho = np.outer(psi, psi.conj())
ppt_results = all_bipartition_PPT_checks(rho, dims=[2]*N)
print("Results of PPT criteria for each partition of 4-qubit cluster state:")
for part, is_ppt in ppt_results.items():
    print(f"Partition {part}: PPT = {is_ppt}")

is_gme = is_genuine_multipartite_entangled(psi, N)
print("Is 4-qubit cluster state genuine multipartite entangled:", is_gme)    

# Star graph adjacency matrix
#（中心 0 連到 1,2,3,4，其餘不連）
N = 5
adj = np.zeros((N, N), dtype=int)
for i in range(1, N):
    adj[0, i] = adj[i, 0] = 1

psi = graph_state(adj)
rho = np.outer(psi, psi.conj())

ppt_results = all_bipartition_PPT_checks(rho, dims=[2]*N)
print("Results of PPT criteria for each partition of 5-qubit star graph state:")
for part, is_ppt in ppt_results.items():
    print(f"Partition {part}: PPT = {is_ppt}")

is_gme = is_genuine_multipartite_entangled(psi, N)
print("Is 5-qubit star graph state genuine multipartite entangled:", is_gme)

Results of PPT criteria for each partition of GHZ state ：
Partition ((0,), (1, 2)): PPT = False
Partition ((1,), (0, 2)): PPT = False
Partition ((2,), (0, 1)): PPT = False
Results of PPT criteria for each partition of 4-qubit cluster state:
Partition ((0,), (1, 2, 3)): PPT = False
Partition ((1,), (0, 2, 3)): PPT = False
Partition ((2,), (0, 1, 3)): PPT = False
Partition ((3,), (0, 1, 2)): PPT = False
Partition ((0, 1), (2, 3)): PPT = False
Partition ((0, 2), (1, 3)): PPT = False
Partition ((0, 3), (1, 2)): PPT = False
Partition ((1, 2), (0, 3)): PPT = False
Partition ((1, 3), (0, 2)): PPT = False
Partition ((2, 3), (0, 1)): PPT = False
Is 4-qubit cluster state genuine multipartite entangled: True
Results of PPT criteria for each partition of 5-qubit star graph state:
Partition ((0,), (1, 2, 3, 4)): PPT = False
Partition ((1,), (0, 2, 3, 4)): PPT = False
Partition ((2,), (0, 1, 3, 4)): PPT = False
Partition ((3,), (0, 1, 2, 4)): PPT = False
Partition ((4,), (0, 1, 2, 3)): PPT = False
P

#### (c) Horodecki Bound Entangled State

In [5]:
# Horodecki 3x3 Bound Entangled State
def horodecki_bound_entangled_state(a=0.5):
    # 3x3 system, 9x9 matrix
    rho = np.zeros((9,9), dtype=np.float64)
    idx = lambda i,j: 3*i + j
    # Fill the matrix
    for i in range(3):
        rho[idx(i,i), idx(i,i)] = a
    for i in range(3):
        for j in range(3):
            if i != j:
                rho[idx(i,j), idx(i,j)] = a
    for i in range(3):
        for j in range(3):
            if i == j:
                rho[idx(i,i), idx(j,j)] += a
    # Diagonal last element
    rho[8,8] = 1
    # Cross terms
    for i in range(3):
        rho[idx(i,i), idx((i+1)%3,(i+1)%3)] = a
        rho[idx((i+1)%3,(i+1)%3), idx(i,i)] = a
    # Normalization
    rho = rho / (8*a + 1)
    return rho

rho = horodecki_bound_entangled_state(a=0.5)
dims = [3,3]
print("Horodecki bound entangled state PPT:", is_separable_PPT(rho, dims))

Horodecki bound entangled state PPT: True


### 2. optimal_PPT_witness

The optimal_entanglement_witness SDP only requires $W$ to be Hermitian, positive semidefinite, and $\mathrm{Tr}[W]=1$ (i.e., $W$ is a valid density matrix). This means it can only detect states $\rho$ that are not positive semidefinite—but all physical quantum states are positive semidefinite by definition.
Therefore, this witness cannot detect any entanglement in physical states. It is just a mathematical example, not a practical entanglement witness.

In contrast, the PPT witness (as implemented in optimal_PPT_witness) requires $W^{T_B} \succeq 0$, i.e., $W$ is in the dual cone of the PPT cone. This allows the SDP to detect all states that are outside the PPT cone (i.e., NPT entangled states), which includes all entangled states detectable by the PPT criterion.

However, even the PPT witness cannot detect PPT entangled states (bound entanglement), such as the Horodecki state. Only more advanced methods (e.g., higher levels of the DPS hierarchy) can detect these.



#### (a) Bell state、Horodecki state

In [6]:
from entanglement_analysis import optimal_PPT_witness, partial_transpose_cvx
# Example: Find optimal PPT witness for Bell state
psi_ent = (np.kron([1,0],[1,0]) + np.kron([0,1],[0,1]))/np.sqrt(2)
rho_ent = np.outer(psi_ent, psi_ent.conj())
dims = [2,2]

W, witness_val = optimal_PPT_witness(rho_ent, dims)
print("Optimal PPT witness value for Bell state:", witness_val)

# Example: Find optimal PPT witness for Horodecki bound entangled state
rho_ent = horodecki_bound_entangled_state(a=0.5)
dims = [3,3]

W, witness_val = optimal_PPT_witness(rho_ent, dims)
print("Optimal PPT witness value for Horodecki bound entangled state:", witness_val)

SDP status: optimal
Witness value Tr(W ρ): -0.5000008959445141
Optimal PPT witness value for Bell state: -0.5000008959445141
SDP status: optimal
Witness value Tr(W ρ): -1.3370179694632833e-07
Optimal PPT witness value for Horodecki bound entangled state: -1.3370179694632833e-07


/usr/local/Caskroom/miniconda/base/envs/qiskit-env/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain_utils.py:41: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(
/usr/local/Caskroom/miniconda/base/envs/qiskit-env/lib/python3.10/site-packages/scs/__init__.py:83: UserWarning: Converting A to a CSC (compressed sparse column) matrix; may take a while.
  warn(


### optimal_entanglement_witness (General Witness)

**Constraints:**
- $W$ is Hermitian
- $W \succeq 0$ (positive semidefinite)
- $\mathrm{Tr}[W] = 1$

Mathematically, this only requires $W$ to be a density matrix (positive semidefinite, trace = 1).

**Detection capability:** This can only detect cases where $\rho$ is not positive semidefinite (which does not occur for physical quantum states). For all physical density matrices (which are positive semidefinite), this witness cannot detect entanglement.

**Nature:** This is just a mathematical example, not a practical entanglement witness.

### optimal_PPT_witness (PPT Witness)

**Constraints:**
- $W$ is Hermitian
- $W^{T_B} \succeq 0$ (partial transpose with respect to subsystem B is also positive semidefinite)
- $\mathrm{Tr}[W] = 1$

Mathematically, this requires $W$ to belong to the dual cone of the PPT cone, and it can detect all states that are outside the PPT cone.

**Detection capability:** This can detect all entangled states that are detectable by the PPT criterion (i.e., all NPT entangled states, but not bound entanglement).

**Nature:** This is a practical and commonly used numerical entanglement witness.

You can use `sdp_witness_all_bipartitions_PPT` to automatically compute the optimal witness value for all bipartitions of a multipartite state, observe and compare the results.

Compare the performance of the witness for different types of states (separable, entangled, biseparable, GME). Try designing your own witness, or use SDP to generate one automatically.

## SDP (Semi-Definite Programming)

SDP is a mathematical optimization tool that minimizes or maximizes an objective function subject to matrix positive semidefinite constraints (e.g., $X \succeq 0$ means all eigenvalues of $X$ are non-negative).
- **Well-posed constraints:** Constraints that can be directly written as mathematical expressions, such as $X$ being positive semidefinite or $\mathrm{Tr}[X] = 1$, are easily handled by SDP solvers like cvxpy.

**Why the separable cone cannot be simply described by SDP:**
The separable cone is the set of all separable states, defined as:
$$ \sigma = \sum_i p_i \, \rho_A^{(i)} \otimes \rho_B^{(i)} $$
This set is mathematically very complex and cannot be fully described by a finite set of simple matrix inequalities or linear equations. In other words, you cannot "enclose" all separable states using only a finite number of simple mathematical constraints.

## SDP: Theory and Numerical Workflow

### 1. The Challenge of Separability Criteria
The set of separable states $\mathcal{S}$ is defined as all $\sigma = \sum_i p_i \rho_A^{(i)} \otimes \rho_B^{(i)}$. $\mathcal{S}$ is a convex cone, but cannot be exactly described by a finite set of linear or semidefinite constraints, so it cannot be directly checked by SDP.

### 2. PPT Cone (Positive Partial Transpose)
The PPT cone $\mathcal{P}$ contains all $\rho$ such that $\rho^{T_B} \succeq 0$. $\mathcal{S} \subset \mathcal{P}$, but $\mathcal{P}$ also contains some entangled states (bound entangled states). The PPT condition can be directly written as an SDP constraint, making it the most common relaxation.

### 3. DPS Hierarchy (Doherty-Parrilo-Spedalieri)
The DPS hierarchy is a sequence of increasingly strict SDP relaxations that gradually approach $\mathcal{S}$. The core idea: if $\rho$ is separable, it must have $k$-extendibility (can be symmetrically extended $k$ times on a subsystem). As $k\to\infty$, the hierarchy converges to the true separable cone.

### 4. SDP Witness Design
For any cone $C$ (e.g., PPT cone, DPS cone), a Hermitian operator $W$ in the dual cone $C^*$ with $\mathrm{Tr}[W\rho]<0$ certifies that $\rho\notin C$. The SDP goal: find $W$ in $C^*$ that minimizes $\mathrm{Tr}[W\rho]$; if the result is $<0$, then $\rho$ is not in $C$.

### Numerical Implementation Workflow
1. **PPT SDP Implementation**
   - Variable: $W$ (Hermitian)
   - Constraints: $\mathrm{Tr}[W]=1$, $W^{T_B} \succeq 0$
   - Objective: Minimize $\mathrm{Tr}[W\rho]$
   - Tools: cvxpy, QETLAB, etc.
2. **DPS SDP Implementation ($k$-extendibility)**
   - Variable: $\tilde{\rho}$ ($k$-extension of $\rho$)
   - Constraints: $\tilde{\rho} \succeq 0$, $\mathrm{Tr}_{2,\ldots,k}[\tilde{\rho}] = \rho$, $\tilde{\rho}$ is symmetric, $\tilde{\rho}^{T_{B_j}} \succeq 0$ for each copy
   - Objective: Feasibility check or witness design
   - Higher $k$ increases computational cost exponentially
3. **Summary**
   - Choose a relaxation cone (PPT or DPS $k$-extendible cone)
   - Write SDP constraints for the cone
   - Design the witness SDP (find $W$ in the dual cone minimizing $\mathrm{Tr}[W\rho]$)
   - Solve numerically (cvxpy, QETLAB, etc.)
   - If witness expectation $<0$, $\rho$ is entangled.

### Implementation: 2-Extendibility (DPS Hierarchy $k=2$) for Bipartite Systems

**Core concept:**
A bipartite state $\rho_{AB}$ is $k$-extendible (with respect to subsystem $B$) if there exists a larger density matrix $\tilde{\rho}_{AB_1B_2...B_k}$ such that:
- $\tilde{\rho}$ is symmetric under permutations of $B_1, ..., B_k$
- Tracing out $B_2, ..., B_k$ returns the original $\rho_{AB}$:
$$ \mathrm{Tr}_{B_2,...,B_k}[\tilde{\rho}] = \rho_{AB} $$

**Physical meaning:**
If $\rho_{AB}$ is separable, it can be "cloned" into $k$ symmetric copies of $B$ without violating quantum mechanics. Entangled states cannot be extended in this way.

**DPS hierarchy:**
- $k=1$: equivalent to the PPT cone
- $k=2,3,...$: each higher level imposes stricter constraints, excluding more entangled states
- As $k\to\infty$, the hierarchy converges to the separable cone

**Intuitive analogy:**
- Separable states: you can symmetrically clone subsystem $B$ many times, and all copies are identical and physical.
- Entangled states: you cannot do this, as it would violate the structure of quantum mechanics (not directly the no-cloning theorem, but a related principle).

Given a density matrix $\rho_{AB}$ of dimension $d_A \times d_B$, we consider 2-extendibility with respect to subsystem $B$.

### Objective

Given $\rho_{AB}$, determine whether there exists an extension $\tilde{\rho}_{AB_1B_2}$ such that:

$\tilde{\rho} \succeq 0$

$\mathrm{Tr}_{B_2}[\tilde{\rho}] = \rho_{AB}$

$\tilde{\rho}$ is symmetric under permutation of $B_1$ and $B_2$ (permutation symmetric)

In [7]:
import cvxpy as cp
import numpy as np
from entanglement_analysis import is_2_extendible


def partial_trace_B2(rho, dA, dB):
    # rho: (dA*dB*dB, dA*dB*dB)
    # trace over the last B (B2)
    rho = rho.reshape((dA, dB, dB, dA, dB, dB))
    # trace over axis 2 and 5 (B2)
    return np.einsum('ijklpq->ijlp', rho).reshape((dA*dB, dA*dB))

def symmetrize_B1B2(rho, dA, dB):
    # symmetrize over B1<->B2
    rho = rho.reshape((dA, dB, dB, dA, dB, dB))
    rho_swap = np.swapaxes(rho, 1, 2)
    rho_swap = np.swapaxes(rho_swap, 4, 5)
    rho_sym = (rho + rho_swap) / 2
    return rho_sym.reshape((dA*dB*dB, dA*dB*dB))


# 測試 Horodecki 3x3 态
rho_horo = horodecki_bound_entangled_state(a=0.5)
dA = dB = 3
print("Horodecki 3x3 态 2-extendible?", is_2_extendible(rho_horo, dA, dB))

/usr/local/Caskroom/miniconda/base/envs/qiskit-env/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


Horodecki 3x3 态 2-extendible? False


## Classical vs Quantum Correlations: Definitions and Mathematical Equivalence

### 1. What is "Correlation"?
Correlation describes whether the measurement outcomes of two subsystems, A and B, are synchronized or related. In classical systems, correlations can be explained by pre-agreed strategies or random sampling. In quantum systems, entanglement leads to correlations that cannot be simulated by any classical means and are only explained by quantum mechanics.

### 2. Classical Correlation (No Entanglement)
**Example:**
$$ \rho = \frac{1}{2} \left( |00\rangle\langle 00| + |11\rangle\langle 11| \right) $$
This is a separable state with no entanglement. Measuring A and B in the $z$-basis always yields the same result (both 0 or both 1), which can be explained by a pre-agreed strategy.

#### Mathematical Definition (Classical Probability)
Suppose there are two random variables $A$ and $B$ with joint probability distribution $P(a, b)$. The expectation value of their correlation is:
$$ \langle AB \rangle_{\text{classical}} = \sum_{a, b} a b P(a, b) $$
If $A$ and $B$ are independent, $P(a, b) = P_A(a) P_B(b)$, so $\langle AB \rangle = \langle A \rangle \langle B \rangle$.

### 3. Quantum Correlation (Entanglement)
**Example:**
$$ |\Phi^+\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle) $$
This is the famous Bell state, which is entangled. Measuring A and B in the $z$-basis also yields synchronized results, but if you change the measurement basis (e.g., $x$-basis), the outcomes remain synchronized. This cannot be explained by any classical pre-agreement. Bell inequality violations show that the correlations exceed classical limits—this is quantum correlation, unique to entangled states.

#### Mathematical Definition (Quantum Separable State)
A separable state:
$$ \rho = \sum_i p_i \rho_A^{(i)} \otimes \rho_B^{(i)} $$
For observables $A$ and $B$, the expectation value is:
$$ \langle A \otimes B \rangle_\rho = \mathrm{Tr}[\rho (A \otimes B)] = \sum_i p_i \mathrm{Tr}[\rho_A^{(i)} A] \mathrm{Tr}[\rho_B^{(i)} B] $$
This is:
$$ \langle A \otimes B \rangle_\rho = \sum_i p_i \langle A \rangle_{\rho_A^{(i)}} \langle B \rangle_{\rho_B^{(i)}} $$

### 4. Equivalence and Distinction
The above formula is mathematically identical to a classical probability mixture: $p_i$ is the probability of sampling, $\langle A \rangle_{\rho_A^{(i)}}$ and $\langle B \rangle_{\rho_B^{(i)}}$ are the averages for each subsystem. Thus, all correlations in separable states can be simulated by classical probability models—there is no "beyond classical" quantum correlation.

In contrast, entangled states exhibit correlations that cannot be explained by any classical model. For these states, measurements in different bases can still yield synchronized outcomes, and violations of Bell inequalities demonstrate the presence of genuine quantum correlations.

### 5. Intuitive Analogy
- **Classical correlation:** Like two people each holding a slip of paper with "0" or "1" written on it, pre-agreed. No matter how you ask, they can only answer according to the paper.
- **Quantum correlation:** No slips of paper, but no matter how you ask (even if you decide at the last moment), their answers are mysteriously synchronized, beyond any classical agreement—only quantum mechanics can explain this.

### 6. Mathematical Expression of Correlation
In quantum information, the most common mathematical definition of correlation between two subsystems is the expectation value $\langle A \otimes B \rangle$:
$$ \langle A \otimes B \rangle = \mathrm{Tr}[\rho (A \otimes B)] $$
where $\rho$ is the two-body density matrix, $A$ and $B$ are observables for each subsystem, and $A \otimes B$ means measuring both simultaneously.

If A and B are independent, $\langle A \otimes B \rangle = \langle A \rangle \langle B \rangle$. If there is correlation, $\langle A \otimes B \rangle$ differs from $\langle A \rangle \langle B \rangle$.
A common example: $A = Z$, $B = Z$, so $\langle Z \otimes Z \rangle$ measures $z$-axis correlation; $A = X$, $B = X$ gives $x$-axis correlation.
You can also define the correlation coefficient: $C_{AB} = \langle A \otimes B \rangle - \langle A \rangle \langle B \rangle$, which measures the pure correlation after subtracting the individual averages.

In [1]:
import numpy as np

# Pauli matrices
I = np.eye(2)
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])

# Projection operators
# |0><0|, |1><1|, |+><+|, |-><-> (where |+> = (|0> + |1>)/sqrt(2), |-> = (|0> - |1>)/sqrt(2))
P0 = np.array([[1,0],[0,0]])
P1 = np.array([[0,0],[0,1]])
Pp = np.array([[0.5,0.5],[0.5,0.5]])   # |+><+|
Pm = np.array([[0.5,-0.5],[-0.5,0.5]]) # |-><-|

# Separable state
# ρ_sep = 0.5 * |0><0| ⊗ |
rho_sep = 0.5 * np.kron(P0, P0) + 0.5 * np.kron(P1, P1)
# Bell State
# ρ_ent = |ψ><ψ| where |ψ> = (|00> + |11>)/sqrt(2)
psi_ent = (np.kron([1,0],[1,0]) + np.kron([0,1],[0,1]))/np.sqrt(2)
rho_ent = np.outer(psi_ent, psi_ent.conj())

def correl(rho, A, B):
    # Compute <A⊗B>
    return np.trace(rho @ np.kron(A, B)).real

print("=== Measure the Z-axis correlation <Z⊗Z> ===")
print("Separable state:", correl(rho_sep, Z, Z))
print("Entangled state  :", correl(rho_ent, Z, Z))

print("\n=== Measure the X-axis correlation <X⊗X> ===")
print("Separable state:", correl(rho_sep, X, X))
print("Entangled state  :", correl(rho_ent, X, X))

=== Measure the Z-axis correlation <Z⊗Z> ===
Separable state: 1.0
Entangled state  : 0.9999999999999998

=== Measure the X-axis correlation <X⊗X> ===
Separable state: 0.0
Entangled state  : 0.9999999999999998
